In [1]:
# Use only GPU 1
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# Allow loading crc_ml scripts as modules
import sys
from pathlib import Path
module_path = Path('.').resolve().parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))

In [3]:
# Bypass memory error
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

In [4]:
# Load modules
import tensorflow.keras.backend as K
import pandas as pd

from src.config import SLIDE_DIR
from src.datagens import WSIDatagen, DataSource
from src.training.models import ContextNASNetModel, NASNetModel

In [ ]:
ckpt = Path('/home/matejg/Project/archive_crc_ml/models/checkpoints/2cf8a974-dcee-499f-8953-86655f556f8d/early.hdf5')
input_shape = (96,96,3)
context_size = 3
dropout = 0.5
learning_rate = 0.0005
output_size = 1
trainable = True

In [ ]:
nasnet = NASNetModel(input_shape, dropout, learning_rate, output_size=output_size, trainable=True)

In [ ]:
# Modify NASNET into ContextNASNET - SUCCESS
'''for _ in range(8):
    inp = tf.keras.layers.Input((96,96,3))
    out = nasnet.model.get_layer('NASNet')(inp)
    out1 = nasnet.model.get_layer('global_max_pooling2d')(out)
    out2 = nasnet.model.get_layer('global_average_pooling2d')(out)
    out3 = nasnet.model.get_layer('flatten')(out)
    out = nasnet.model.get_layer('concatenate_4')([out1, out2, out3])
    out = nasnet.model.get_layer('dropout')(out)
    out = nasnet.model.get_layer('dense')(out)
    new_outputs.append(out)
    new_inputs.append(inp)
out = tf.keras.layers.Concatenate(axis=-1)(nasnet.model.outputs + new_outputs)
out = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=None)(out)
new_model = tf.keras.Model(nasnet.model.inputs + new_inputs, out)
new_model.load_weights(str(ckpt))'''

In [ ]:
# Modify NASNET into BatchContextNASNET
def backend_reshape(x):
    return K.reshape(x, (-1, 9))

out = nasnet.model.get_layer('dense').output
out = tf.keras.layers.Lambda(backend_reshape, output_shape=(9,))(out)
out = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=None)(out)
new_model = tf.keras.Model(nasnet.model.input, out)
new_model.load_weights(str(ckpt))

In [ ]:
# Load default params for NASNet
import json
with (Path('.').resolve().parent / 'configs' / 'wsi.json').open('r') as json_in:
    params = json.load(json_in)
    
# Checkpoint is from model trained on tiles from Level 1 and Context Size 3
params['data']['level'] = 1
params['data']['context_size'] = 3

# Use smaller batch_size because the shape will become 
# (batch_size * context_size * context_size, input_shape, input_shape, channels)
params['batch_size'] = 8


In [ ]:
# Create datasource with the test image TP-2019_2623-12-1.gz
test_ds = DataSource()
test_ds.coord_maps = [Path('/home/matejg/Project/crc_ml/data/processed/Prostata/level1/context3/t96px/all/coord_maps/TP-2019_2623-12-1.gz')]

In [ ]:
df = pd.read_pickle('/home/matejg/Project/crc_ml/data/processed/Prostata/level1/context3/t96px/all/coord_maps/TP-2019_2623-12-1.gz')
len(df)

In [ ]:
-(-10545 // 8)

In [ ]:
# Create datagen and obtain sequential generator
datagen = WSIDatagen(**params['data'], slide_dir=SLIDE_DIR, batch_size=params['batch_size'])
test_gen = datagen.get_sequential_generator((96,96), test_ds)

In [ ]:
# Test - make sure the input data are correct
#
# Every 9 consecutive images corresponds to a same macrotile
#
# The order is: LT, T, RT, L, C, R, BL, B, BR
# (L - left, R - Right, T - Top, B - Bottom, C - Center)
X, y = test_gen[0]
print(X.shape)

In [ ]:
# Make predictions for the Center images
res = new_model.predict_generator(test_gen, verbose=1)

In [ ]:
# Threshold predictions and save them to df
threshold = 0.5
df = pd.read_pickle(Path('/home/matejg/Project/crc_ml/data/processed/Prostata/level1/context3/t96px/all/coord_maps/TP-2019_2623-12-1.gz'))
df['predict'] = (res >= threshold).astype(np.int)

# Calculate metrics for given thresholded results
trues = df[df['tumor_tile'] == 1] 
TP = sum(trues['tumor_tile'] == trues['predict'])
FN = sum(trues['tumor_tile'] != trues['predict'])

falses = df[df['tumor_tile'] == 0] 
TN = sum(falses['tumor_tile'] == falses['predict'])
FP = sum(falses['tumor_tile'] != falses['predict'])

print('True positive:   {}\nTrue negatives:  {}\nFalse positive:  {}\nFalse negatives: {}\n'.format(TP,TN,FP,FN))

prec = TP / (TP + FP)
recc = TP / (TP + FN)
f1 = 2*prec*recc/(prec+recc)
print('Precision: {:.2%}\nRecall:    {:.2%}\n'.format(prec, recc))

from sklearn.metrics import roc_auc_score
print('AUC@{:.1f}: {:.2%}'.format(threshold, roc_auc_score(df['tumor_tile'].ravel(), res.ravel())))